# AI based Natural disaster analysis

### Importing Neccessary Libraries

In [ ]:
import numpy as np#used for numerical analysis
import tensorflow #open source used for both ML and DL for computation
from tensorflow.keras.models import Sequential #it is a plain stack of layers
from tensorflow.keras import layers #A layer consists of a tensor-in tensor-out computation function
#Dense layer is the regular deeply connected neural network layer
from tensorflow.keras.layers import Dense,Flatten
#Faltten-used fot flattening the input or change the dimension
from tensorflow.keras.layers import Conv2D,MaxPooling2D #Convolutional layer
#MaxPooling2D-for downsampling the image
from keras.preprocessing.image import ImageDataGenerator


In [ ]:
tensorflow.__version__

'2.8.0'

In [ ]:
tensorflow.keras.__version__

'2.8.0'

### Image Data Agumentation

In [ ]:
#setting parameter for Image Data agumentation to the training data
train_datagen = ImageDataGenerator(rescale=1./255,shear_range=0.2,zoom_range=0.2,horizontal_flip=True)
#Image Data agumentation to the testing data
test_datagen=ImageDataGenerator(rescale=1./255)

### Loading our data and performing data agumentation

In [ ]:
#performing data agumentation to train data
x_train = train_datagen.flow_from_directory('../dataset/train_set',target_size=(64, 64),batch_size=5,
                                            color_mode='rgb',class_mode='categorical')
#performing data agumentation to test data
x_test = test_datagen.flow_from_directory('../dataset/test_set',target_size=(64, 64),batch_size=5,
                                            color_mode='rgb',class_mode='categorical') 

Found 742 images belonging to 4 classes.
Found 198 images belonging to 4 classes.


In [ ]:
print(x_train.class_indices)#checking the number of classes

{'Cyclone': 0, 'Earthquake': 1, 'Flood': 2, 'Wildfire': 3}


In [ ]:
print(x_test.class_indices)#checking the number of classes

{'Cyclone': 0, 'Earthquake': 1, 'Flood': 2, 'Wildfire': 3}


In [ ]:
from collections import Counter as c
c(x_train .labels)

Counter({0: 220, 1: 156, 2: 198, 3: 168})

### Creating the model

In [ ]:
# Initializing the CNN
classifier = Sequential()

# First convolution layer and pooling
classifier.add(Conv2D(32, (3, 3), input_shape=(64, 64, 3), activation='relu'))
classifier.add(MaxPooling2D(pool_size=(2, 2)))
# Second convolution layer and pooling
classifier.add(Conv2D(32, (3, 3), activation='relu'))
# input_shape is going to be the pooled feature maps from the previous convolution layer
classifier.add(MaxPooling2D(pool_size=(2, 2)))

# Flattening the layers
classifier.add(Flatten())

# Adding a fully connected layer
classifier.add(Dense(units=128, activation='relu'))
classifier.add(Dense(units=4, activation='softmax')) # softmax for more than 2



In [ ]:
classifier.summary()#summary of our model

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 62, 62, 32)        896       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 31, 31, 32)       0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 29, 29, 32)        9248      
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 14, 14, 32)       0         
 2D)                                                             
                                                                 
 flatten (Flatten)           (None, 6272)              0         
                                                                 
 dense (Dense)               (None, 128)               8

### Compiling the model

In [ ]:
# Compiling the CNN
# categorical_crossentropy for more than 2
classifier.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy']) 

## Fitting the model

In [ ]:
classifier.fit_generator(
        generator=x_train,steps_per_epoch = len(x_train),
        epochs=20, validation_data=x_test,validation_steps = len(x_test))# No of images in test set

C:\Users\HP\AppData\Local\Temp/ipykernel_7732/549542485.py:1: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  classifier.fit_generator(


Epoch 1/20
149/149 [==============================] - 136s 876ms/step - loss: 1.1739 - accuracy: 0.4542 - val_loss: 1.1279 - val_accuracy: 0.5253
Epoch 2/20
149/149 [==============================] - 72s 487ms/step - loss: 0.7935 - accuracy: 0.6685 - val_loss: 0.8564 - val_accuracy: 0.6616
Epoch 3/20
149/149 [==============================] - 72s 486ms/step - loss: 0.7362 - accuracy: 0.7129 - val_loss: 0.7500 - val_accuracy: 0.7020
Epoch 4/20
149/149 [==============================] - 73s 490ms/step - loss: 0.6701 - accuracy: 0.7547 - val_loss: 1.3990 - val_accuracy: 0.5152
Epoch 5/20
149/149 [==============================] - 91s 613ms/step - loss: 0.6437 - accuracy: 0.7493 - val_loss: 0.7660 - val_accuracy: 0.7424
Epoch 6/20
149/149 [==============================] - 78s 528ms/step - loss: 0.5630 - accuracy: 0.7938 - val_loss: 0.7220 - val_accuracy: 0.7576
Epoch 7/20
149/149 [==============================] - 95s 639ms/step - loss: 0.5100 - accuracy: 0.8261 - val_loss: 1.4338 - val_a

### Saving our model

In [ ]:
# Save the model
classifier.save('disaster.h5')

In [ ]:
model_json = classifier.to_json()
with open("model-bw.json", "w") as json_file:
    json_file.write(model_json)

### Predicting our results

In [ ]:
from tensorflow.keras.models import load_model
from keras.preprocessing import image
model = load_model("disaster.h5") #loading the model for testing

In [ ]:
img = image.load_img(r"..\dataset\test_set\Cyclone\921.jpg",grayscale=False,
                     target_size= (64,64))#loading of the image
x = image.img_to_array(img)#image to array
x = np.expand_dims(x,axis = 0)#changing the shape
pred = model.predict(x)#predicting the classes
pred = np.argmax(pred) #getting index of higher probability value
pred

0

In [ ]:
index=['Cyclone','Earthquake','Flood','Wildfire']
result=str(index[pred])
result

'Cyclone'